In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fbprophet import Prophet
import yfinance as yf
import datetime


In [7]:

def runHyper(tickerList):
    best_params = {}
    best_log_likelihood = -1000000000
    today = datetime.date.today()

    for ticker in tickerList:
        df = yf.download(ticker, start='2015-01-01', end= today, progress=False)
        df = df.rename(columns={'Date': 'ds', 'Adj Close': 'y'})
        df['ds'] = pd.to_datetime(df['ds'])

        # Split the data into training and testing sets
        train_df = df[df['ds'] < '2021-01-01']
        test_df = df[df['ds'] >= '2021-01-01']

        # Perform hyperparameter tuning
        for growth in ['linear', 'logistic']:
            for changepoint_prior_scale in [0.01, 0.05, 0.1, 0.5, 1]:
                for n_changepoints in [5, 10, 25, 50, 100]:
                    model = Prophet(
                        growth=growth,
                        changepoint_prior_scale=changepoint_prior_scale,
                        n_changepoints=n_changepoints
                    )
                    model.fit(train_df)

                    future = model.make_future_dataframe(periods=365)
                    forecast = model.predict(future)
                    log_likelihood = model.log_likelihood_baseline_only()
                    if log_likelihood > best_log_likelihood:
                        best_log_likelihood = log_likelihood
                        best_params = {
                            'growth': growth,
                            'changepoint_prior_scale': changepoint_prior_scale,
                            'n_changepoints': n_changepoints
                        }
                        best_model = model
                        best_forecast = forecast

    # Plot the forecast using the best model
    plt.figure(figsize=(10, 7))
    best_model.plot(best_forecast, xlabel='Date', ylabel='Adj Close')
    plt.title(f'{ticker} Stock Price Forecast (Best Model)')
    plt.show()

tickerList = ['AAPL', 'MSFT', 'GOOG', 'AMZN']
runHyper(tickerList)

KeyError: 'ds'